In [ ]:
!pip install byaldi together pdf2image
!sudo apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
pdf_path = "/content/drive/MyDrive/Task/Tables.pdf"


In [ ]:
from pathlib import Path
from byaldi import RAGMultiModalModel

# Initialize the ColQwen2 model
model = RAGMultiModalModel.from_pretrained("vidore/colqwen2-v0.1")

# Set the index name
index_name = "table_index"  # Choose a name for your index

# Define the path to your PDF file in Google Drive
pdf_path = Path("/content/drive/MyDrive/Task/Tables.pdf")  # Update with your actual file path

# Index the PDF and store embeddings
model.index(
    input_path=pdf_path,  # Path to your uploaded PDF
    index_name=index_name,  # Index name
    store_collection_with_index=True,  # Store base64 images along with vectors
    overwrite=True  # Overwrite the index if it already exists
)


Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Added page 1 of document 0 to index.
Added page 2 of document 0 to index.
Added page 3 of document 0 to index.
Added page 4 of document 0 to index.
Added page 5 of document 0 to index.
Added page 6 of document 0 to index.
Added page 7 of document 0 to index.
Added page 8 of document 0 to index.
Added page 9 of document 0 to index.
Added page 10 of document 0 to index.
Added page 11 of document 0 to index.
Added page 12 of document 0 to index.
Added page 13 of document 0 to index.
Added page 14 of document 0 to index.
Added page 15 of document 0 to index.
Added page 16 of document 0 to index.
Added page 17 of document 0 to index.
Added page 18 of document 0 to index.
Added page 19 of document 0 to index.
Index exported to .byaldi/table_index
Index exported to .byaldi/table_index


{0: '/content/drive/MyDrive/Task/Tables.pdf'}

In [ ]:
# Define the query you want to search for in your document
query = "What is the U.S. GDP by industry for 2015?"  # Replace with your actual query

# Perform the search using the model
results = model.search(query, k=5)  # Adjust the 'k' value for the number of results you want

# Display the results
print(f"Search results for '{query}':")
for result in results:
    print(f"Doc ID: {result.doc_id}, Page: {result.page_num}, Score: {result.score}")

# Test completion message
print("Test completed successfully!")


Search results for 'What is the U.S. GDP by industry for 2015?':
Doc ID: 0, Page: 6, Score: 21.375
Doc ID: 0, Page: 7, Score: 19.5
Doc ID: 0, Page: 8, Score: 19.25
Doc ID: 0, Page: 11, Score: 18.125
Doc ID: 0, Page: 12, Score: 14.625
Test completed successfully!


In [ ]:
%%timeit
model.search(query, k=5)

173 ms ± 2.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
import os
from together import Together

# Set up your API key
api_key = '7bfd8d20ad127325a928a14996f3846c837b389273565552a0f63927823f2124'  # Replace with your actual API key
client = Together(api_key=api_key)

# Define your query for the PDF content (unemployment info based on degree type)
query = "What is the unemployment information based on type of degree from page 2?"

# Search for the relevant content in your index (adjusted for your PDF content)
results = model.search(query, k=1)

# Extract the base64 image of the relevant page (page 2 in this case)
returned_page = results[0].base64

# Set up the request to the Llama-3.2 90B Vision model
response = client.chat.completions.create(
  model="meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": query},  # The query you are asking
        {
          "type": "image_url",  # Pass the base64 encoded image of the page
          "image_url": {
            "url": f"data:image/jpeg;base64,{returned_page}",
          },
        },
      ],
    }
  ],
  max_tokens=300,  # Adjust max tokens if needed
)

# Print the answer from the Llama model
print(response.choices[0].message.content)


This information is not present on page 2, it is on page 1. However, the unemployment rate for different degrees are as follows:
Doctoral degree: 2.2%
Professional degree: 2.3%
Master's degree: 3.4%
Bachelor's degree: 4.0%
Associate's degree: 5.4%
Some college, no degree: 7.0%
High school diploma: 7.5%
Less than a high school diploma: 11.0%


In [22]:
!pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 76.4 MB/s eta 0:00:00


In [24]:
!pip install faiss-gpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.2 MB/s eta 0:00:00


In [6]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

pdf_path = "/content/drive/MyDrive/Task/Tables.pdf"
pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text[:1000])  # Print the first 1000 characters for preview


Tables, Charts, and
Graphs
with Examples from History, Economics,
Education, Psychology, Urban Affairs and
Everyday Life
REVISED: MICHAEL LOLKUS 2018Tables, Charts, and
Graphs Basics We use charts and graphs to visualize data.
 This data can either be generated data, data gathered from
an experiment, or data collected from some source.
 A picture tells a thousand words so it is not a surprise that
many people use charts and graphs when explaining data.Types of Visual
Representations of DataTable of Yearly U.S. GDP by
Industry (in millions of dollars)
Source: U.S. Bureau of Labor Statistics
Year 2010 2011 2012 2013 2014 2015
All Industries 26093515 27535971 28663246 29601191 30895407 31397023
Manufacturing 4992521 5581942 5841608 5953299 6047477 5829554
Finance,
Insurance, Real
4522451 4618678 4797313 5031881 5339678 5597018
Estate, Rental,
Leasing
Arts,
Entertainment,
Recreation, 964032 1015238 1076249 1120496 1189646 1283813
Accommodation,
and Food Service
Other 15614511 16320113 1

In [8]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [11]:
import nltk
nltk.download('punkt')

def segment_text(text):
    sentences = nltk.sent_tokenize(text)  # Tokenize the text into sentences
    return sentences

chunks = segment_text(pdf_text)
print(chunks[:5])  # Print first 5 chunks


['Tables, Charts, and\nGraphs\nwith Examples from History, Economics,\nEducation, Psychology, Urban Affairs and\nEveryday Life\nREVISED: MICHAEL LOLKUS 2018Tables, Charts, and\nGraphs Basics\uf075 We use charts and graphs to visualize data.', '\uf075 This data can either be generated data, data gathered from\nan experiment, or data collected from some source.', '\uf075 A picture tells a thousand words so it is not a surprise that\nmany people use charts and graphs when explaining data.Types of Visual\nRepresentations of DataTable of Yearly U.S. GDP by\nIndustry (in millions of dollars)\nSource: U.S. Bureau of Labor Statistics\nYear 2010 2011 2012 2013 2014 2015\nAll Industries 26093515 27535971 28663246 29601191 30895407 31397023\nManufacturing 4992521 5581942 5841608 5953299 6047477 5829554\nFinance,\nInsurance, Real\n4522451 4618678 4797313 5031881 5339678 5597018\nEstate, Rental,\nLeasing\nArts,\nEntertainment,\nRecreation, 964032 1015238 1076249 1120496 1189646 1283813\nAccommodati

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
def extract_tables(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        tables = []
        for page in pdf.pages:
            tables.extend(page.extract_tables())
    return tables

tables = extract_tables(pdf_path)
print(tables[0])  # Preview the first table


[['Year', '2010', '2011', '2012', '2013', '2014', '2015'], ['All Industries', '26093515', '27535971', '28663246', '29601191', '30895407', '31397023'], ['Manufacturing', '4992521', '5581942', '5841608', '5953299', '6047477', '5829554'], ['Finance,\nInsurance, Real\nEstate, Rental,\nLeasing', '4522451', '4618678', '4797313', '5031881', '5339678', '5597018'], ['Arts,\nEntertainment,\nRecreation,\nAccommodation,\nand Food Service', '964032', '1015238', '1076249', '1120496', '1189646', '1283813'], ['Other', '15614511', '16320113', '16948076', '17495515', '18318606', '18686638']]


In [13]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Convert text chunks into vector embeddings
embeddings = model.encode(chunks)
print(f"Embeddings shape: {len(embeddings)} chunks, {embeddings[0].shape[0]} dimensions per chunk.")


Embeddings shape: 37 chunks, 384 dimensions per chunk.


In [14]:
import faiss
import numpy as np

# Convert embeddings to a numpy array
embedding_matrix = np.array(embeddings)

# Create a FAISS index
index = faiss.IndexFlatL2(embedding_matrix.shape[1])  # Use L2 distance for similarity

# Add embeddings to the index
index.add(embedding_matrix)

# Now, the embeddings are stored and indexed for similarity search


**Query Handling**

In [15]:
query = "What is the GDP of the U.S. for 2015?"
query_embedding = model.encode([query])  # Embed the query
print(f"Query embedding: {query_embedding}")


Query embedding: [[ 3.07771675e-02 -3.03827901e-03  7.82589894e-03  4.82906699e-02
  -9.00156237e-03 -5.29249050e-02 -1.99551657e-02 -1.55069670e-02
   2.59969421e-02 -1.06154177e-02  5.01680300e-02 -5.01686893e-02
  -3.77514251e-02 -7.49706803e-03 -8.55468735e-02  3.13869081e-02
   3.00396439e-02 -6.59852624e-02  4.43415232e-02 -3.68005671e-02
   3.16533558e-02  5.25094382e-03  3.25285532e-02 -5.28640449e-02
  -1.76585093e-02  1.54615371e-02 -4.17847745e-02 -1.05376601e-01
  -6.77188411e-02 -3.13952100e-03 -6.15213402e-02 -9.63207148e-03
   2.95881219e-02 -5.18690795e-02  1.39797963e-02 -3.69107313e-02
   2.15790067e-02  1.08134355e-02  5.57171367e-02 -1.76403839e-02
  -7.25849951e-03 -7.34323487e-02 -3.07888370e-02  8.13480746e-03
  -1.59368865e-04  3.19719799e-02  1.03034452e-02  9.83656049e-02
   8.26258492e-03  1.14026554e-01 -3.49699263e-03  1.29006296e-01
   2.24063918e-02  2.20986325e-02  8.64144340e-02  4.98683099e-03
  -4.04096860e-03  3.50253470e-02  3.34842391e-02  6.577751

In [16]:
# Perform similarity search for the query embedding
D, I = index.search(query_embedding, k=5)  # k=5 to get top 5 most similar chunks

# Output the top 5 most similar chunks
print("Top 5 most similar chunks to the query:")
for idx in I[0]:
    print(f"Chunk {idx+1}: {chunks[idx]}")  # Output the corresponding chunks


Top 5 most similar chunks to the query:
Chunk 16: 2015 GDP (in trillions of dollars)
Other
Arts, entertainment, recreation, accommodation, and food
services
Finance, insurance, real estate, rental, and leasing
Manufacturing
All industries
0 5 10 15 20 25 30 35
Dollars• The graph below is called a line graph.
Chunk 14: • 2015 GDP for all industries was $31.397023; looking at the graph,
the bar for all industries is just above $30.
Chunk 5: • If total GDP in 2015 is the entire pie, then
manufacturing makes up 19% of that pie and finance
makes up 18%.
Chunk 7: 2015 U.S. GDP (in millions of dollars)
Manufacturing
19%
Finance, insurance, real
estate, rental, and
leasing
18% Arts, entertainment,
59%
recreation,
accommodation, and
food services
Other
4% Pie charts can be misleading when the slices do not
correspond with the percent contribution to the whole pie.
Chunk 21: 
In our example, we compared components of US GDP.


In [18]:
!pip install transformers


In [20]:
from together import Together

# Set up your API key
api_key = '7bfd8d20ad127325a928a14996f3846c837b389273565552a0f63927823f2124'  # Replace with your actual API key
client = Together(api_key=api_key)

# Define your query and context
query = "What is the GDP of the U.S. for 2015?"
context = "The GDP of the U.S. in 2015 was approximately 18 trillion USD."

# Set up the request to the LLaMA model
response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo",
    messages=[
        {"role": "user", "content": query},
        {"role": "assistant", "content": context}
    ],
    max_tokens=150
)

# Print the answer from the model
print(response.choices[0].message.content)


According to the Bureau of Economic Analysis (BEA), the nominal GDP of the United States in 2015 was $18,036.6 billion.


In [29]:
# Print the full API response to check the structure
print(response)


id='8f34c7467a24ce77-SIN' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1734416124 model='meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo' choices=[ChatCompletionChoicesData(index=0, logprobs=None, seed=None, finish_reason=<FinishReason.StopSequence: 'stop'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='To be more precise, the Gross Domestic Product (GDP) of the United States for 2015 was:\n\n$18,036,648,000,000 (nominal)\n\nThis is according to the Bureau of Economic Analysis (BEA), which is the official source for GDP data in the United States.\n\nNote: The GDP figure is in nominal terms, which means it is not adjusted for inflation.', tool_calls=[]))] prompt=[] usage=UsageData(prompt_tokens=47, completion_tokens=83, total_tokens=130) prompt_logprobs=None


In [27]:
# Check if response contains choices and handle accordingly
if 'choices' in response and len(response['choices']) > 0:
    print(response['choices'][0]['message']['content'])
else:
    print("No valid response received.")


No valid response received.


In [28]:
response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo",
    messages=[
        {"role": "user", "content": query},
        {"role": "assistant", "content": context}
    ],
    max_tokens=200  # Increase token limit if necessary
)


3